# 2023 NFL Playoff Fantasy


## Imports

In [1]:
!pip install nflscraPy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
#import seaborn as sns
import re

import nflscraPy

import math
plt.style.use('fivethirtyeight')
import requests
from lxml import html
from lxml import etree
import openpyxl

from bs4 import BeautifulSoup
from typing import Dict, List

from pandas.plotting import register_matplotlib_converters
pd.set_option('display.max_columns', None)

### Import Rosters

In [4]:
## IMPORT ROSTERS
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eG86C0rtDOxG3BFP9bTg9h3Z2yJCqHW_71r4QXVyeNA/edit#gid=203918864')

# Get a Dictionary with each team
teams_dict = worksheet.get_worksheet(6).get_all_records()
print(teams_dict)

# Convert to a DataFrame and case / format the names correctly for PRF
fantasy_rosters = pd.DataFrame.from_dict(teams_dict).drop(columns = {"Timestamp", "Paid?"})
fantasy_rosters = fantasy_rosters.apply(lambda x: x.str.lower().str.title().str.strip())

# Replace Defense names with the team's mascot (for PRF compatability)
d_dict = {"Sf": "49ers", "Buf": "Bills", "Phi": "Eagles", "Cin": "Bengals", "Kc": "Chiefs"}
fantasy_rosters[["D1", "D2"]] = fantasy_rosters[["D1", "D2"]].replace(d_dict)
fantasy_rosters

[{'What is your name?': 'John McGonigle', 'QB1': 'Brock Purdy', 'QB2': 'Josh Allen', 'K1': 'Robbie Gould', 'K2': 'Tyler Bass', 'D1': 'SF', 'D2': 'BUF', 'Pos1': 'Christian McCaffrey', 'Pos2': 'George Kittle', 'Pos3': 'Stefon Diggs', 'Pos4': 'Dawson Knox', 'Pos5': 'A.J. Brown', 'Pos6': 'Jerick McKinnon', 'Pos7': 'Travis Kelce', 'Timestamp': '1/10/2023 12:06:23', 'Paid?': 'Yes'}, {'What is your name?': 'Nick Solit', 'QB1': 'Brock Purdy', 'QB2': 'Josh Allen', 'K1': 'Robbie Gould', 'K2': 'Tyler Bass', 'D1': 'SF', 'D2': 'BUF', 'Pos1': 'Christian McCaffrey', 'Pos2': 'George Kittle', 'Pos3': 'Stefon Diggs', 'Pos4': 'Deebo Samuel', 'Pos5': 'A.J. Brown', 'Pos6': 'Travis Kelce', 'Pos7': 'Austin Ekeler', 'Timestamp': '1/10/2023 13:03:07', 'Paid?': 'Yes'}, {'What is your name?': 'Micah Solit', 'QB1': 'Josh Allen', 'QB2': 'Patrick Mahomes', 'K1': 'Robbie Gould', 'K2': 'Tyler Bass', 'D1': 'SF', 'D2': 'PHI', 'Pos1': 'Christian McCaffrey', 'Pos2': 'Stefon Diggs', 'Pos3': 'Travis Kelce', 'Pos4': 'Ceedee

,What is your name?,QB1,QB2,K1,K2,D1,D2,Pos1,Pos2,Pos3,Pos4,Pos5,Pos6,Pos7
0,John Mcgonigle,Brock Purdy,Josh Allen,Robbie Gould,Tyler Bass,49ers,Bills,Christian Mccaffrey,George Kittle,Stefon Diggs,Dawson Knox,A.J. Brown,Jerick Mckinnon,Travis Kelce
1,Nick Solit,Brock Purdy,Josh Allen,Robbie Gould,Tyler Bass,49ers,Bills,Christian Mccaffrey,George Kittle,Stefon Diggs,Deebo Samuel,A.J. Brown,Travis Kelce,Austin Ekeler
2,Micah Solit,Josh Allen,Patrick Mahomes,Robbie Gould,Tyler Bass,49ers,Eagles,Christian Mccaffrey,Stefon Diggs,Travis Kelce,Ceedee Lamb,Ja'Marr Chase,George Kittle,Joe Mixon
3,Alex Meyers,Josh Allen,Brock Purdy,Tyler Bass,Robbie Gould,Bills,49ers,Devin Singletary,Stefon Diggs,Gabriel Davis,Christian Mccaffrey,Austin Ekeler,Deebo Samuel,Ceedee Lamb
4,Vinay Ayyala,Jalen Hurts,Josh Allen,Jake Elliott,Robbie Gould,Bills,49ers,Christian Mccaffrey,Miles Sanders,Stefon Diggs,A.J. Brown,George Kittle,Dawson Knox,Devonta Smith
5,Michael Bellas,Josh Allen,Brock Purdy,Robbie Gould,Tyler Bass,49ers,Bills,Christian Mccaffrey,Ja'Marr Chase,Deebo Samuel,Stefon Diggs,Justin Jefferson,A.J. Brown,Gabriel Davis
6,Kelly Mcgonigle,Brock Purdy,Josh Allen,Robbie Gould,Tyler Bass,49ers,Bills,Christian Mccaffrey,Deebo Samuel,George Kittle,Dawson Knox,Stefon Diggs,Travis Kelce,A.J. Brown
7,Skip Tague,Jalen Hurts,Josh Allen,Robbie Gould,Tyler Bass,Eagles,Bills,Travis Kelce,Stefon Diggs,A.J. Brown,Miles Sanders,Christian Mccaffrey,Ezekiel Elliott,Jerick Mckinnon
8,The Khoisan People,Brock Purdy,Joe Burrow,Robbie Gould,Evan Mcpherson,49ers,Bengals,Deebo Samuel,George Kittle,Christian Mccaffrey,Brandon Aiyuk,Elijah Mitchell,Ja'Marr Chase,Joe Mixon
9,Tim Tague,Jalen Hurts,Josh Allen,Jake Elliott,Tyler Bass,Eagles,Bills,A.J. Brown,Miles Sanders,Stefon Diggs,Dawson Knox,Travis Kelce,Jerick Mckinnon,Tee Higgins


## TODO 

- Functionalize everything -- input url output all the tables needed
- Extend Functions so that we can have each week's scoring in the same DF
- Write to Excel, or App (since everything is stored in a dict, that should be pretty easy

## Weekly
- Input new games urls
- Run the code on Monday
- check 2 pt conversions
- check defensive scores


### Requests for API Guy
- Update roster info so its not just starters
- Add scorers of Pass PATs

In [5]:
## Replace Names with weird capitalization
fantasy_rosters = fantasy_rosters.replace({"Christian Mccaffrey": "Christian McCaffrey",
                                "Evan Mcpherson": "Evan McPherson",
                                "Ceedee Lamb": "CeeDee Lamb"})

In [6]:
pos_counts = fantasy_rosters[["Pos1", "Pos2", "Pos3", "Pos4", "Pos5", "Pos6", "Pos7"]].apply(pd.value_counts).sum(axis=1).sort_values(ascending=False)
pos_counts

Christian McCaffrey    21.0
Stefon Diggs           20.0
Travis Kelce           15.0
George Kittle          14.0
A.J. Brown             12.0
Ja'Marr Chase          10.0
Deebo Samuel            9.0
Gabriel Davis           6.0
Dawson Knox             6.0
Jerick Mckinnon         6.0
Tee Higgins             5.0
Joe Mixon               5.0
Justin Jefferson        5.0
Miles Sanders           4.0
Austin Ekeler           4.0
Devin Singletary        3.0
Devonta Smith           3.0
CeeDee Lamb             3.0
Elijah Mitchell         2.0
Brandon Aiyuk           2.0
Tony Pollard            1.0
Cole Beasley            1.0
Mike Evans              1.0
Dallas Goedert          1.0
Ezekiel Elliott         1.0
Isaiah Pacheco          1.0
dtype: float64

In [7]:
qb_counts = fantasy_rosters[["QB1", "QB2"]].apply(pd.value_counts).sum(axis=1).sort_values(ascending=False)
qb_counts

Josh Allen         18
Brock Purdy        13
Jalen Hurts         6
Joe Burrow          5
Patrick Mahomes     4
dtype: int64

In [8]:
k_counts = fantasy_rosters[["K1", "K2"]].apply(pd.value_counts).sum(axis=1).sort_values(ascending=False)
k_counts

Robbie Gould       19
Tyler Bass         16
Evan McPherson      5
Harrison Butker     3
Jake Elliott        3
dtype: int64

In [9]:
d_counts = fantasy_rosters[["D1", "D2"]].apply(pd.value_counts).sum(axis=1).sort_values(ascending=False)
d_counts

49ers      19.0
Bills      17.0
Eagles      7.0
Bengals     2.0
Chiefs      1.0
dtype: float64

## Make Fantasy Points Data

### Cleaning Functions

In [13]:
def extract_scores_and_scorers(raw_scoring_df):
  points_df = raw_scoring_df

  # Extract the Relevant columns from the import DF
  home_team, away_team = points_df["tm_name"][0], points_df["opp_name"][0]
  points_df = points_df.loc[:, ["event_date", "quarter", "tm_score", "opp_score","scoring_team", "description"]]
  points_df = points_df.rename(columns = {"event_date": "game_date",
                                          "tm_score" : home_team, 
                                          "opp_score": away_team, 
                                          "description": "score_txt"})
  points_df["quarter"] = points_df["quarter"].astype("int")

  # Strip & Move PAT info to new column 
  points_df['PAT_txt'] = points_df["score_txt"].str.extract(r'\((.*?)\)').astype('str').replace({"nan": "None"})
  points_df["score_txt"] = points_df["score_txt"].str.replace(r'\(.*?\)', '', regex=True)

  ## Calculate PAT_values -- Copy from old code?
  points_df.loc[:, "PAT_value"] = 2
  points_df.loc[points_df["PAT_txt"].str.contains("kick"), "PAT_value"] = 1
  points_df.loc[points_df["PAT_txt"].str.contains("failed"), "PAT_value"] = 0
  points_df.loc[points_df["PAT_txt"].str.contains("None"), "PAT_value"] = 0

  # Extract Distance into new column
  points_df['score_distance'] = points_df['score_txt'].apply(lambda x: (re.search(r'\d+', x).group() if re.search(r'\d+', x) is not None else None))
  points_df['score_distance'] = points_df['score_distance'].astype(int)

  # TODO: Extract PAT_val (0, 1, 2) based on "failed" or NaN, "kick", neither
  #points_df['PAT_value']

  # One-Hot Encode Play Type
  points_df['rush'] = points_df['score_txt'].str.contains('rush')*1
  points_df['pass'] = points_df['score_txt'].str.contains('pass')*1
  points_df['FG'] = points_df['score_txt'].str.contains('field goal')*1
  points_df['return'] = points_df['score_txt'].str.contains('return')*1
  points_df['safety'] = points_df['score_txt'].str.contains('safety')*1

  ## Add New cols that might be blank if that score doesn't occur
  points_df[["rusher", "catcher", "passer", "FG_kicker", 
            "PAT_kicker", "PAT_rusher", "PAT_catcher", "PAT_passer"]
            # ] = pd.DataFrame([[None, None, None, None, 
            #                    None, None, None, None]], index=points_df.index)     
            ] = pd.DataFrame([[" - ", " - ", " - ", " - ", 
                              " - ", " - ", " - ", " - "]], index=points_df.index)
            
  ## Create columns that track who the scorers were
  ## Note PAT_catcher & PAT_passer aren't available from the API
  points_df['rusher'] = points_df['score_txt'].apply(lambda x: re.findall(r"^([a-zA-z.\s\'\-]+)\d", x.strip())[0] if re.search(r'rush', x) is not None else "")
  points_df['catcher'] = points_df['score_txt'].apply(lambda x: re.findall(r"^([a-zA-z.\s\'\-]+)\d", x.strip())[0] if re.search(r'pass', x) is not None else "")
  points_df['passer'] = points_df['score_txt'].apply(lambda x: re.findall(r"pass from (.*)", x.strip())[0] if re.search(r'pass', x) is not None else "")
  points_df['FG_kicker'] = points_df['score_txt'].apply(lambda x: re.findall(r"^([a-zA-z.\s\'\-]+)\d", x.strip())[0] if re.search(r'field goal', x) is not None else "")
  points_df["PAT_kicker"] = points_df["PAT_txt"].apply(lambda x: re.findall(r"^(.*?[\w\' ])\bkick", x.strip())[0] if re.search(r'kick', x) is not None else "")
  points_df["PAT_rusher"] = points_df["PAT_txt"].apply(lambda x: re.findall(r"^(.*?) run", x.strip())[0] if re.search(r'run', x) is not None else "")

  # if points_df["PAT_txt"].str.contains("two point pass successful").any():
  #   print(f"WARNING: 2 pt pass found in {home_team, away_team} game") 
  # else:
  #   print(f"No 2 pt passes in {home_team, away_team} game")

  return points_df

In [14]:
def calculate_fantasy_points(scores_and_scorers_df):
  df = scores_and_scorers_df

  ## Calculate the value for the Offensive TDs
  df["o_td_fpts"] = ((5 * df["rush"] + (df["score_distance"]) // 10) * df["rush"]) + (
    (5 * df["pass"] + (df["score_distance"]) // 10) * df["pass"])

  ## Calculate the value for Offensive 2-pt conversions
  df["o_pat_fpts"] = np.where(df['PAT_value'] == 2, df['PAT_value'], 0)

  ## Calculate the value for kickers (PAT + FGs)
  df["k_fpts"] = ((df["FG"] * df['score_distance']) // 10) + (
    np.where(df['PAT_value'] < 2, df['PAT_value'], 0))
  df.loc[(df["score_distance"] > 55) & (df["FG"] == 1), "k_fpts"] = df["k_fpts"] + (df["score_distance"] - 55)

  ## Calculate the value for Defensive scores (TD + Safety)
  df["d_fpts"] = ((5 * df["return"] + (df['score_distance']) // 10) * df["return"]) + (
    df["safety"] * 2
  )

  return df

In [15]:
def pat2_finder(scoring_df):
  for key in scoring_df.keys():
    # wc_scoring[print(key)]
    pat2_df = scoring_df[key]
    pat2_ind = pat2_df.index[pat2_df["PAT_value"] == 2].to_list()
    pat2_rows = pat2_df[pat2_df["PAT_value"] == 2]
    display(pat2_rows)
    #display(pat2_rows)

In [16]:
def pass_2pt_assigner(scoring_df : pd.DataFrame, game_str : str , catcher_str : str, passer_str : str):
  pat2_df = scoring_df[game_str]
  pat2_ind = pat2_df.index[pat2_df["PAT_value"] == 2].to_list()
  pat2_rows = pat2_df[pat2_df["PAT_value"] == 2]
  pat2_rows.loc[pat2_ind, "PAT_catcher"] = catcher_str.strip()
  pat2_rows.loc[pat2_ind, "PAT_passer"] = passer_str.strip()
  scoring_df[game_str].loc[pat2_ind, : ] = pat2_rows.loc[pat2_ind, :]
  return scoring_df


In [17]:
def point_per_player_extractor(one_game_df : pd.DataFrame, columns_to_extract : list, points_column : str):
  # Create an empty dataframe to store the new values
  df = pd.DataFrame(columns=['player', 'points'])
  try:
    # Iterate through each row of the original dataframe
    for index, row in one_game_df.iterrows():
        # Iterate through each column in columns_to_extract
        for col in columns_to_extract:
            # Check if the value is not None
            if row[col] not in [None, " - "]:
                # Append the player name and corresponding point to the new dataframe
                #new_df = new_df.append({'player': row[col], 'points':row[points_column]}, ignore_index=True)
                df = pd.concat([df, pd.DataFrame({'player': row[col], 'points':row[points_column]}, index=[0])])
    df = df[df["player"] != ""].groupby("player").sum()
    df = df.sort_values(by="points", ascending=False).reset_index()
    return df
  except:
    away, home = one_game_df.columns[3], one_game_df.columns[2]
    print(f"In {away, home} game, Failed to assign points from {columns_to_extract}")
    return df

In [18]:
def fpts_aggregator(one_game_df : pd.DataFrame, round : str):
  o_tds = point_per_player_extractor(one_game_df, ["rusher", "catcher", "passer"], "o_td_fpts")
  o_pats = point_per_player_extractor(one_game_df, ["PAT_rusher", "PAT_catcher", "PAT_passer"], "o_pat_fpts")
  k_fpts = point_per_player_extractor(one_game_df, ["FG_kicker", "PAT_kicker"], "k_fpts") 
  d_fpts = point_per_player_extractor(one_game_df, ["scoring_team"], "d_fpts")
  d_fpts = d_fpts[d_fpts["points"] > 0] 

  fpts_df = pd.concat([o_tds, k_fpts, o_pats, d_fpts])
  fpts_df["player"] = fpts_df["player"].str.strip()
  fpts_df = fpts_df.rename(columns = {"points": f"{round}_points"})
  fpts_df = fpts_df.groupby("player").sum(numeric_only=True
      ).sort_values(by = f"{round}_points", ascending = False).reset_index()
  
  return fpts_df

In [19]:
def put_all_fpts_in_one_df(round_scoring, round : str):
  fantasy_points = pd.DataFrame(columns=['player', f"{round}_points"])
  for team, scoring_df in round_scoring.items():
    game_fpts = fpts_aggregator(scoring_df, round)
    fantasy_points = pd.concat([fantasy_points, game_fpts])

  fantasy_points = fantasy_points.sort_values(by=f"{round}_points", 
                                              ascending=False).reset_index(drop=True)
                                            
  return fantasy_points



In [20]:
def wc_make_fpts_teams_and_standings_dicts(points_str : str = "points"):
  fpts_teams_dict = {}
  standings_dict = {}
  for ind, name in enumerate(fantasy_rosters["What is your name?"]):
    #print(name)
    roster_ind = (fantasy_rosters["What is your name?"] == name)
    team = pd.DataFrame(fantasy_rosters[roster_ind]).reset_index(drop=True).transpose()
    team = team.rename(columns =  {0: "player"})
    team = team.merge(wc_fpts, how = 'left', on = "player").fillna(0)
    team[points_str] = team[points_str].astype(dtype='int64')

    total = team.loc[1: 13, points_str].sum()
    team.loc[14, :] = ["Total", total]
    fpts_teams_dict[name] = team
    standings_dict[name] = team.loc[14, points_str]
  return fpts_teams_dict, standings_dict

In [21]:
def make_fpts_teams_and_standings_dicts(points_str : str , round : str, fpts_df : pd.DataFrame):
  ## If we're inputting WC data, we need to start from scratch (the fantasy_rosters) df
  for name, team in fpts_teams_dict.items():
    team = team.merge(fpts_df, how = 'left', on = "player").fillna(0)
    team[points_str] = team[points_str].astype(dtype='int64')

    total = team.loc[1: 13, points_str].sum()
    team.loc[14, points_str] = total
    

    team.loc[14, "player"] = team.loc[14, team.columns[1:]].sum()
    standings_dict[name] = team.loc[14, "player"]

    fpts_teams_dict[name] = team


  return fpts_teams_dict, standings_dict

In [22]:
#### Pull NFL Data from API

#### Pull NFL Data from API

##### Super WC Weekend

In [23]:
wc_games_url_dict = {"sea_sf" : "https://www.pro-football-reference.com/boxscores/202301140sfo.htm",
                     "lac_jac" : "https://www.pro-football-reference.com/boxscores/202301140jax.htm",
                     "mia_buf" : "https://www.pro-football-reference.com/boxscores/202301150buf.htm",
                     "nyg_min" : "https://www.pro-football-reference.com/boxscores/202301150min.htm",
                     "bal_cin" : "https://www.pro-football-reference.com/boxscores/202301150cin.htm",
                     "dal_tb" : "https://www.pro-football-reference.com/boxscores/202301160tam.htm"
}
## Pull raw scoring plays from nflscraPy API
wc_data = {teams: nflscraPy._gamelog_scoring(url) for teams, url in wc_games_url_dict.items()}
## Break down into playtype & players
wc_scoring = {teams: extract_scores_and_scorers(df) for teams, df in wc_data.items()}
## Visually check 2 point plays & Team Names!
pat2_finder(wc_scoring)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301140sfo.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301140jax.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301150buf.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processi

,game_date,quarter,49ers,Seahawks,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
8,2023-01-14,4,31,17,49ers,Elijah Mitchell 7 yard pass from Brock Purdy,two point pass successful,2,7,0,1,0,0,0,,Elijah Mitchell,Brock Purdy,,,,-,-


,game_date,quarter,Jaguars,Chargers,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
9,2023-01-14,4,28,30,Jaguars,Christian Kirk 9 yard pass from Trevor Lawrence,Trevor Lawrence run,2,9,0,1,0,0,0,,Christian Kirk,Trevor Lawrence,,,Trevor Lawrence,-,-


,game_date,quarter,Bills,Dolphins,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
6,2023-01-15,2,17,17,Dolphins,Mike Gesicki 7 yard pass from Skylar Thompson,two point pass successful,2,7,0,1,0,0,0,,Mike Gesicki,Skylar Thompson,,,,-,-


,game_date,quarter,Vikings,Giants,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


,game_date,quarter,Bengals,Ravens,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
4,2023-01-15,3,17,10,Bengals,Joe Burrow 1 yard rush,two point pass successful,2,1,1,0,0,0,0,Joe Burrow,,,,,,-,-


,game_date,quarter,Buccaneers,Cowboys,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
6,2023-01-16,4,14,31,Buccaneers,Cameron Brate 8 yard pass from Tom Brady,two point pass successful,2,8,0,1,0,0,0,,Cameron Brate,Tom Brady,,,,-,-


check ESPN for other scoring data:
https://www.espn.com/nfl/game/_/gameId/401438002

In [24]:
## Add 2 Point Pass-Catchers in the WC round
round_scoring = wc_scoring
round_scoring = pass_2pt_assigner(scoring_df = round_scoring, game_str = "sea_sf", catcher_str = "George Kittle", passer_str = "Brock Purdy")
round_scoring = pass_2pt_assigner(scoring_df = round_scoring, game_str = "mia_buf", catcher_str = "Tyreek Hill", passer_str = "Skylar Thompson")
round_scoring = pass_2pt_assigner(scoring_df = round_scoring, game_str = "bal_cin", catcher_str = "Tee Higgins", passer_str = "Joe Burrow")
round_scoring = pass_2pt_assigner(scoring_df = round_scoring, game_str = "dal_tb", catcher_str = "Mike Evans", passer_str = "Tom Brady")

## Calculate each scoring play's Fantasy points
wc_fantasy_points = {teams: calculate_fantasy_points(scoring_df) for teams, scoring_df in round_scoring.items()}

In [25]:
wc_scoring["bal_cin"]

,game_date,quarter,Bengals,Ravens,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer,o_td_fpts,o_pat_fpts,k_fpts,d_fpts
0,2023-01-15,1,3,0,Bengals,Evan McPherson 39 yard field goal,None,0,39,0,0,1,0,0,,,,Evan McPherson,,,-,-,0,0,3,0
1,2023-01-15,2,9,0,Bengals,Ja'Marr Chase 7 yard pass from Joe Burrow,Evan McPherson kick failed,0,7,0,1,0,0,0,,Ja'Marr Chase,Joe Burrow,,Evan McPherson,,-,-,5,0,0,0
2,2023-01-15,2,9,7,Ravens,J.K. Dobbins 2 yard pass from Tyler Huntley,Justin Tucker kick,1,2,0,1,0,0,0,,J.K. Dobbins,Tyler Huntley,,Justin Tucker,,-,-,5,0,1,0
3,2023-01-15,2,9,10,Ravens,Justin Tucker 22 yard field goal,None,0,22,0,0,1,0,0,,,,Justin Tucker,,,-,-,0,0,2,0
4,2023-01-15,3,17,10,Bengals,Joe Burrow 1 yard rush,two point pass successful,2,1,1,0,0,0,0,Joe Burrow,,,,,,Tee Higgins,Joe Burrow,5,2,0,0
5,2023-01-15,3,17,17,Ravens,Demarcus Robinson 41 yard pass from Tyler Hunt...,Justin Tucker kick,1,41,0,1,0,0,0,,Demarcus Robinson,Tyler Huntley,,Justin Tucker,,-,-,9,0,1,0
6,2023-01-15,4,24,17,Bengals,Sam Hubbard 98 yard fumble return,Evan McPherson kick,1,98,0,0,0,1,0,,,,,Evan McPherson,,-,-,0,0,1,14


In [26]:
wc_fpts = put_all_fpts_in_one_df(wc_scoring, "wc")
wc_fpts

<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()


In ('Giants', 'Vikings') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']


,player,wc_points
0,Brock Purdy,29
1,Dak Prescott,29
2,Trevor Lawrence,25
3,Josh Allen,17
4,Robbie Gould,16
5,Geno Smith,15
6,D.K. Metcalf,15
7,Tom Brady,15
8,Kirk Cousins,15
9,Tyler Huntley,14


In [27]:
fpts_teams_dict, standings_dict= wc_make_fpts_teams_and_standings_dicts(points_str = "wc_points")


### Divisional Round

In [28]:
div_games_url_dict = {"jac_kc" : "https://www.pro-football-reference.com/boxscores/202301210kan.htm",
                     "nyg_phi" : "https://www.pro-football-reference.com/boxscores/202301210phi.htm",
                     "cin_buf" : "https://www.pro-football-reference.com/boxscores/202301220buf.htm",
                     "dal_sf" : "https://www.pro-football-reference.com/boxscores/202301220sfo.htm",
}
                     
## Pull raw scoring plays from nflscraPy API
div_data = {teams: nflscraPy._gamelog_scoring(url) for teams, url in div_games_url_dict.items()}
## Break down into playtype & players
div_scoring = {teams: extract_scores_and_scorers(df) for teams, df in div_data.items()}
## Visually check 2 point plays & Team Names!
pat2_finder(div_scoring)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301210kan.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301210phi.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301220buf.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processi

,game_date,quarter,Chiefs,Jaguars,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


,game_date,quarter,Eagles,Giants,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


,game_date,quarter,Bills,Bengals,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


,game_date,quarter,49ers,Cowboys,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


In [29]:
## Add 2 Point Pass-Catchers in the WC round (there are none in 2023)
round_scoring = div_scoring

## Calculate each scoring play's Fantasy points
div_fantasy_points = {teams: calculate_fantasy_points(scoring_df) for teams, scoring_df in round_scoring.items()}

In [30]:
div_fantasy_points["cin_buf"]

,game_date,quarter,Bills,Bengals,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer,o_td_fpts,o_pat_fpts,k_fpts,d_fpts
0,2023-01-22,1,0,7,Bengals,Ja'Marr Chase 28 yard pass from Joe Burrow,Evan McPherson kick,1,28,0,1,0,0,0,,Ja'Marr Chase,Joe Burrow,,Evan McPherson,,-,-,7,0,1,0
1,2023-01-22,1,0,14,Bengals,Hayden Hurst 15 yard pass from Joe Burrow,Evan McPherson kick,1,15,0,1,0,0,0,,Hayden Hurst,Joe Burrow,,Evan McPherson,,-,-,6,0,1,0
2,2023-01-22,2,7,14,Bills,Josh Allen 1 yard rush,Tyler Bass kick,1,1,1,0,0,0,0,Josh Allen,,,,Tyler Bass,,-,-,5,0,1,0
3,2023-01-22,2,7,17,Bengals,Evan McPherson 28 yard field goal,None,0,28,0,0,1,0,0,,,,Evan McPherson,,,-,-,0,0,2,0
4,2023-01-22,3,10,17,Bills,Tyler Bass 25 yard field goal,None,0,25,0,0,1,0,0,,,,Tyler Bass,,,-,-,0,0,2,0
5,2023-01-22,3,10,24,Bengals,Joe Mixon 1 yard rush,Evan McPherson kick,1,1,1,0,0,0,0,Joe Mixon,,,,Evan McPherson,,-,-,5,0,1,0
6,2023-01-22,4,10,27,Bengals,Evan McPherson 20 yard field goal,None,0,20,0,0,1,0,0,,,,Evan McPherson,,,-,-,0,0,2,0


In [31]:
div_fpts = put_all_fpts_in_one_df(div_scoring, "div")
div_fpts

<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()
<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()
<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()


In ('Jaguars', 'Chiefs') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']
In ('Giants', 'Eagles') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']
In ('Bengals', 'Bills') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']
In ('Cowboys', '49ers') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']


<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()


,player,div_points
0,Jalen Hurts,16
1,Robbie Gould,14
2,Harrison Butker,13
3,Joe Burrow,13
4,Riley Patterson,10
5,Travis Kelce,10
6,Patrick Mahomes,10
7,Kenneth Gainwell,8
8,Jake Elliott,8
9,Evan McPherson,7


### Championship Round

In [32]:
champ_games_url_dict = {"sf_phi" : "https://www.pro-football-reference.com/boxscores/202301290phi.htm",
                     "cin_kc" : "https://www.pro-football-reference.com/boxscores/202301290kan.htm",
}
                     
## Pull raw scoring plays from nflscraPy API
champ_data = {teams: nflscraPy._gamelog_scoring(url) for teams, url in champ_games_url_dict.items()}
## Break down into playtype & players
champ_scoring = {teams: extract_scores_and_scorers(df) for teams, df in champ_data.items()}
## Visually check 2 point plays & Team Names!
pat2_finder(champ_scoring)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301290phi.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202301290kan.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,game_date,quarter,Eagles,49ers,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


,game_date,quarter,Chiefs,Bengals,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer


In [33]:
## Add 2 Point Pass-Catchers in the WC round (there are none in 2023)
round_scoring = champ_scoring

## Calculate each scoring play's Fantasy points
champ_fantasy_points = {teams: calculate_fantasy_points(scoring_df) for teams, scoring_df in round_scoring.items()}

In [34]:
champ_fantasy_points["cin_kc"]

,game_date,quarter,Chiefs,Bengals,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer,o_td_fpts,o_pat_fpts,k_fpts,d_fpts
0,2023-01-29,1,3,0,Chiefs,Harrison Butker 43 yard field goal,None,0,43,0,0,1,0,0,,,,Harrison Butker,,,-,-,0,0,4,0
1,2023-01-29,2,6,0,Chiefs,Harrison Butker 24 yard field goal,None,0,24,0,0,1,0,0,,,,Harrison Butker,,,-,-,0,0,2,0
2,2023-01-29,2,6,3,Bengals,Evan McPherson 30 yard field goal,None,0,30,0,0,1,0,0,,,,Evan McPherson,,,-,-,0,0,3,0
3,2023-01-29,2,13,3,Chiefs,Travis Kelce 14 yard pass from Patrick Mahomes,Harrison Butker kick,1,14,0,1,0,0,0,,Travis Kelce,Patrick Mahomes,,Harrison Butker,,-,-,6,0,1,0
4,2023-01-29,2,13,6,Bengals,Evan McPherson 23 yard field goal,None,0,23,0,0,1,0,0,,,,Evan McPherson,,,-,-,0,0,2,0
5,2023-01-29,3,13,13,Bengals,Tee Higgins 27 yard pass from Joe Burrow,Evan McPherson kick,1,27,0,1,0,0,0,,Tee Higgins,Joe Burrow,,Evan McPherson,,-,-,7,0,1,0
6,2023-01-29,3,20,13,Chiefs,Marquez Valdes-Scantling 19 yard pass from Pat...,Harrison Butker kick,1,19,0,1,0,0,0,,Marquez Valdes-Scantling,Patrick Mahomes,,Harrison Butker,,-,-,6,0,1,0
7,2023-01-29,4,20,20,Bengals,Samaje Perine 2 yard rush,Evan McPherson kick,1,2,1,0,0,0,0,Samaje Perine,,,,Evan McPherson,,-,-,5,0,1,0
8,2023-01-29,4,23,20,Chiefs,Harrison Butker 45 yard field goal,None,0,45,0,0,1,0,0,,,,Harrison Butker,,,-,-,0,0,4,0


In [35]:
champ_fpts = put_all_fpts_in_one_df(champ_scoring, "champ")
champ_fpts

In ('49ers', 'Eagles') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']
In ('Bengals', 'Chiefs') game, Failed to assign points from ['PAT_rusher', 'PAT_catcher', 'PAT_passer']


<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()
<ipython-input-17-33c64732059c>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df[df["player"] != ""].groupby("player").sum()


,player,champ_points
0,Harrison Butker,12
1,Patrick Mahomes,12
2,Miles Sanders,11
3,Christian McCaffrey,7
4,Jake Elliott,7
5,Evan McPherson,7
6,Joe Burrow,7
7,Tee Higgins,7
8,Boston Scott,6
9,Marquez Valdes-Scantling,6


### Super Bowl

In [36]:
superBowl_url_dict = {"phi_kc" : "https://www.pro-football-reference.com/boxscores/202302120phi.htm",
}            

## Pull raw scoring plays from nflscraPy API
superBowl_data = {teams: nflscraPy._gamelog_scoring(url) for teams, url in superBowl_url_dict.items()}
## Break down into playtype & players
superBowl_scoring = {teams: extract_scores_and_scorers(df) for teams, df in superBowl_data.items()}
## Visually check 2 point plays & Team Names!
pat2_finder(superBowl_scoring)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Processing Game Scoring For: https://www.pro-football-reference.com/boxscores/202302120phi.htm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,game_date,quarter,Chiefs,Eagles,scoring_team,score_txt,PAT_txt,PAT_value,score_distance,rush,pass,FG,return,safety,rusher,catcher,passer,FG_kicker,PAT_kicker,PAT_rusher,PAT_catcher,PAT_passer
10,2023-02-12,4,35,35,Eagles,Jalen Hurts 2 yard rush,Jalen Hurts run,2,2,1,0,0,0,0,Jalen Hurts,,,,,Jalen Hurts,-,-


In [37]:
## Calculate each scoring play's Fantasy points
superBowl_fantasy_points = {teams: calculate_fantasy_points(scoring_df) for teams, scoring_df in superBowl_scoring.items()}
superBowl_fpts = put_all_fpts_in_one_df(superBowl_scoring, "superBowl")
superBowl_fpts

,player,superBowl_points
0,Jalen Hurts,26
1,Patrick Mahomes,16
2,A.J. Brown,9
3,Jake Elliott,9
4,Chiefs,8
5,Harrison Butker,7
6,Travis Kelce,6
7,Isiah Pacheco,5
8,Kadarius Toney,5
9,Skyy Moore,5


### Apply Fantasy Points to each Fantasy team

In [38]:
fpts_teams_dict.keys(), fpts_teams_dict["John Mcgonigle"]

(dict_keys(['John Mcgonigle', 'Nick Solit', 'Micah Solit', 'Alex Meyers', 'Vinay Ayyala', 'Michael Bellas', 'Kelly Mcgonigle', 'Skip Tague', 'The Khoisan People', 'Tim Tague', 'Phillycheese', 'Jared Jensen', 'Ryan Walker', 'Ben Wemer', 'Will Sisca', 'Casey Mcgonigle', 'Troy Nadel', 'Alex Warner', 'Jake Withers', 'Andy Duncan', 'Marky P', 'Jack Withers', 'Will Grant']),
                  player  wc_points
 0        John Mcgonigle        0.0
 1           Brock Purdy       29.0
 2            Josh Allen       17.0
 3          Robbie Gould       16.0
 4            Tyler Bass       10.0
 5                 49ers        0.0
 6                 Bills        0.0
 7   Christian McCaffrey        5.0
 8         George Kittle        2.0
 9          Stefon Diggs        0.0
 10          Dawson Knox        5.0
 11           A.J. Brown        0.0
 12      Jerick Mckinnon        0.0
 13         Travis Kelce        0.0
 14                Total       84.0)

In [39]:
make_fpts_teams_and_standings_dicts("div_points", fpts_df = div_fpts, round = 'div')
make_fpts_teams_and_standings_dicts("champ_points", fpts_df = champ_fpts, round = "champ")
make_fpts_teams_and_standings_dicts("superBowl_points", fpts_df = superBowl_fpts, round = "superBowl")

({'John Mcgonigle':                  player  wc_points  div_points  champ_points  superBowl_points
  0        John Mcgonigle        0.0           0             0                 0
  1           Brock Purdy       29.0           0             0                 0
  2            Josh Allen       17.0           5             0                 0
  3          Robbie Gould       16.0          14             1                 0
  4            Tyler Bass       10.0           3             0                 0
  5                 49ers        0.0           0             0                 0
  6                 Bills        0.0           0             0                 0
  7   Christian McCaffrey        5.0           5             7                 0
  8         George Kittle        2.0           0             0                 0
  9          Stefon Diggs        0.0           0             0                 0
  10          Dawson Knox        5.0           0             0                 0
  11      

In [40]:
standings = pd.DataFrame({k: v for k, v in sorted(standings_dict.items(), key=lambda item: item[1], reverse=True)},index=[0])
standings = standings.transpose().rename(columns = {0 : "Standings"})
standings["Standings"] = standings["Standings"].astype(dtype="int64")
standings

,Standings
Casey Mcgonigle,196
Ryan Walker,194
Troy Nadel,191
Marky P,180
The Khoisan People,177
Andy Duncan,177
Skip Tague,172
Nick Solit,168
Micah Solit,168
Vinay Ayyala,168


In [41]:
standings.index

Index(['Casey Mcgonigle', 'Ryan Walker', 'Troy Nadel', 'Marky P',
       'The Khoisan People', 'Andy Duncan', 'Skip Tague', 'Nick Solit',
       'Micah Solit', 'Vinay Ayyala', 'Will Sisca', 'Jake Withers',
       'Kelly Mcgonigle', 'Tim Tague', 'Jack Withers', 'Ben Wemer',
       'Michael Bellas', 'John Mcgonigle', 'Jared Jensen', 'Alex Meyers',
       'Phillycheese', 'Alex Warner', 'Will Grant'],
      dtype='object')

In [42]:
team_name = "Will Grant"

fpts_teams_dict[team_name]

,player,wc_points,div_points,champ_points,superBowl_points
0,Will Grant,0.0,0,0,0
1,Josh Allen,17.0,5,0,0
2,Jalen Hurts,0.0,16,5,26
3,Tyler Bass,10.0,3,0,0
4,Jake Elliott,0.0,8,7,9
5,Bills,0.0,0,0,0
6,Eagles,0.0,0,0,0
7,Dawson Knox,5.0,0,0,0
8,Cole Beasley,5.0,0,0,0
9,Stefon Diggs,0.0,0,0,0



### Find Best Possible Team

In [43]:
fpts_frames = [
    wc_fpts,
    div_fpts,
    champ_fpts,
    superBowl_fpts
]

In [44]:
total_fpts = (wc_fpts
              .merge(div_fpts, on = "player", how = "outer")
              .merge(champ_fpts, on = "player", how = "outer")
              .merge(superBowl_fpts, on = "player", how = "outer"))

total_fpts = total_fpts.fillna(0)
total_fpts["Total Points"] = total_fpts[total_fpts.columns].sum(axis = 1)
total_fpts = total_fpts.sort_values("Total Points", ascending=False).reset_index(drop=True)
total_fpts

<ipython-input-44-bd4bb6b4240e>:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  total_fpts["Total Points"] = total_fpts[total_fpts.columns].sum(axis = 1)


,player,wc_points,div_points,champ_points,superBowl_points,Total Points
0,Jalen Hurts,0,16,5,26,47
1,Patrick Mahomes,0,10,12,16,38
2,Dak Prescott,29,5,0,0,34
3,Harrison Butker,0,13,12,7,32
4,Joe Burrow,12,13,7,0,32
...,...,...,...,...,...,...
76,Evan Engram,5,0,0,0,5
77,Justin Tucker,4,0,0,0,4
78,Mike Evans,2,0,0,0,2
79,Tyreek Hill,2,0,0,0,2
